In [1]:
using Gurobi, JuMP   #JuMP是一个优化库(可以在里面插入优化的库，比如Gurobi) Import everything 
using DataFrames, GLM
using Plots
using Distributions
using Random
using LinearAlgebra
using DelimitedFiles

┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1342
┌ Warning: Module JSON with build ID 163554486034401 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JSON with build ID 163554486034401 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JSON with build ID 163554486034401 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JSON with build ID 163554486034401 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module 

In [2]:
#rand-均匀分布； randn-正态分布
#Function for simulating data
function Data(n,m,a,b,c,r,s,form="flat",seed=1)
	Random.seed!(seed)   #Random: Support for generating random numbers.   Random.rand!-Function是的，(seed)可以将随机数固定
	#Drawing random decision data
	x = rand(n)  #x是[0,1]之间的随机数，所以我们产生的data points是随机的
	#Drawing random outcome data
	y,y_min,y_max = Outcome(x,a,b,c,r,s,form)
	#Setting all possible values for decisions
    d = collect(0.1:1/m:0.9)
	return x,y,y_min,y_max,d
end

#Function for simulating outcome. 拟合各种函数结果
function Outcome(x,a,b,c,r,s,form="flat")
	n = length(x)
    y = zeros(n)
    #Simulating the random outcome for each random decision
    for i = 1:n
        if form == "flat"
            y[i] = a + s*randn()[1]
        elseif form == "flat-double"
            y[i] = a + s*randn()[1]
            if 0.5 <= x[i] <= 0.75
                y[i] = r*a + s*randn()[1]   #这里有随机项，所以对于相同的x值，y值也有很小的差别
            end
        elseif form == "linear"
            y[i] = a + b*x[i] + s*randn()[1]
        elseif form == "linear-double"
            y[i] = a + b*x[i] + s*randn()[1]
            if 0.5 <= x[i]
                y[i] = r*a + a*(1-r)*x[i] + s*randn()[1]
            end
        elseif form == "quadratic"
            y[i] = a + b*x[i] + c*x[i]^2 + s*randn()[1]
        elseif form == "quadratic-double"
            y[i] = a + b*x[i] + c*x[i]^2 + s*randn()[1]   #此处需要数字而不是数组，故需要将randn() 转化为randn()[1]
            if 0.5 <= x[i]
                y[i] = r*a + (3*a*(1-r)+2*b+c)*x[i] + (-2*a*(1-r)-2*b-c)*x[i]^2 + s*randn()[1]    #r具体是多少？如何使得函数在边界点处相等？可以用公式计算出r的表达式
            end
        elseif form == "newsvendor"
        	demand = a + s*randn()[1]
        	y[i] = -(c*x[i] + r*b*max(demand-x[i],0) + b*max(x[i]-demand,0))
        elseif form == "supplychain-normal"
        	y[i] = x[i] * (quantile(Normal(c,r),(a-x[i])/(a+b))) + s*randn()[1]
        elseif form == "supplychain-pareto"
        	y[i] = x[i] * (quantile(Pareto(c,r),(a-x[i])/(a+b))) + s*randn()[1]
        end		       
    end
    #Scaling the outcome to unit scale
    y_min = minimum(y)
    y_max = maximum(y)
    y = (y - ones(n)*y_min)./(ones(n)*(y_max-y_min))   #将y归一化
	return y,y_min,y_max
end


Outcome (generic function with 2 methods)

In [3]:
#Generating data
x_data,y_data,y_min_data,y_max_data,d_data = Data(n,m,alpha,beta,gamma,rho,sigma,form_data,seed_number)

LoadError: UndefVarError: n not defined

In [4]:
#Function for computing result  
function Result(y_min,y_max,x,a,b,c,r,form="flat")
	#Computing the expected outcome for given decision
    if form == "flat"
        y = a
    elseif form == "flat-double"
        y = a
        if 0.5 <= x <= 0.75
            y = r*a               ### What is the value of r??????
        end
    elseif form == "linear"
        y = a + b*x
    elseif form == "linear-double"
        y = a + b*x
        if 0.5 <= x
            y = r*a + a*(1-r)*x
        end
    elseif form == "quadratic"
        y = a + b*x + c*x^2
    elseif form == "quadratic-double"
        y = a + b*x + c*x^2
        if 0.5 <= x
            y = r*a + (3*a*(1-r)+2*b+c)*x + (-2*a*(1-r)-2*b-c)*x^2
        end
    elseif form == "newsvendor"
    	demand = a
    	y = -(c*x + r*b*max(demand-x,0) + b*max(x-demand,0))
    elseif form == "supplychain-normal"
        y = x * (quantile(Normal(c,r),(a-x)/(a+b)))
    elseif form == "supplychain-pareto"
        y = x * (quantile(Pareto(c,r),(a-x)/(a+b)))
    end
    #Scaling the outcome to unit scale
    y = (y-y_min)/(y_max-y_min)
	return y
end

Result (generic function with 2 methods)

In [5]:
#Function for separately estimating and optimizing
function ETO(x,y,d,form="linear",showoutput=false)
    n = length(x)
    m = length(d)
    #Estimating outcome model
    df = DataFrame(outcome=y,decision=x)
    if form == "linear"
		ols = lm(@formula(outcome~decision),df)
		a,b = coef(ols)
		c = 0
    elseif form == "quadratic"
		ols = lm(@formula(outcome~decision+decision^2),df)
		a,b,c = coef(ols)
	end
	#Defining optimization model
    model = Model()
	set_optimizer(model, Gurobi.Optimizer)
	set_optimizer_attributes(model, "OutputFlag" => 0)    
	#Defining variables
    @variable(model, z[1:m], Bin)
    #Defining objective
   	@objective(model, Max, sum((a+b*d[j]+c*d[j]^2)*z[j] for j=1:m))
    #Defining constraints
    @constraints(model, begin
        decision, sum(z[j] for j=1:m) == 1
    end)
    #Optimizing model
    optimize!(model)
    obj_val = objective_value(model)
    time_val = solve_time(model)
    a_val = a
    b_val = b
	c_val = c
    z_val = value.(z)
    return obj_val,time_val,a_val,b_val,c_val,z_val
end


ETO (generic function with 3 methods)

In [6]:
#Selecting setup
#setup = ["flat-double",100,0,0,4,1,0.05,1]
setup = ["linear-double",100,100,0,0.75,1,0.05,1]
#setup = ["quadratic-double",100,100,-100,0.5,1,0.05,0.1]
#setup = ["newsvendor",0.5,4,1,0.75,0.1,0.05,0.1]
#setup = ["supplychain-normal",1,1,4,3,0.1,0.05,0.1]
#setup = ["supplychain-pareto",1,1.5,0.25,1,0.1,0.05,0.1]

#Setting parameters    
form_data = setup[1]
alpha = setup[2]
beta = setup[3]
gamma = setup[4]
rho = setup[5]
sigma = setup[6]   #相当于前面的s
kappa = setup[7]
lambda = setup[8]
n = 1000
m = 100
big_M = 1000
seed_numbers = 100

100

In [7]:
#Printing parameter settings
println("Parameter settings:")
println("form_data: ", form_data)
println("alpha: ", alpha)
println("beta: ", beta)
println("gamma: ", gamma)
println("rho: ", rho)
println("sigma: ", sigma)
println("kappa: ", kappa)
println("lambda: ", lambda)
println("n: ", n)
println("m: ", m)
println("seed_numbers: ", seed_numbers)

Parameter settings:
form_data: linear-double
alpha: 100
beta: 100
gamma: 0
rho: 0.75
sigma: 1
kappa: 0.05
lambda: 1
n: 1000
m: 100
seed_numbers: 100


In [8]:
#Generating data
x_data,y_data,y_min_data,y_max_data,d_data = Data(n,m,alpha,beta,gamma,rho,sigma,form_data,seed_numbers)

([0.2601250914736861, 0.19031281518127185, 0.6609109399808133, 0.06719317094984745, 0.9675998379215747, 0.6456910432314067, 0.5459681993995775, 0.5268448425879988, 0.9727545053153894, 0.8681941503861808  …  0.05396706059723866, 0.574222318679148, 0.8630835157915413, 0.9597376251011864, 0.4352254480289155, 0.7363952963887221, 0.6592224743412349, 0.4588690920708318, 0.12446084234043298, 0.5958808066096684], [0.6161264360168017, 0.4882623772490506, 0.08563943021180923, 0.34047368099801983, 0.19120309346451925, 0.08389689451807694, 0.025951033017401304, 0.015284428330698822, 0.1992298133633091, 0.16369570901219696  …  0.297452313496418, 0.04071902586345343, 0.17450165395771053, 0.19310327599446878, 0.8809896369566835, 0.1119970862496957, 0.09660107711362767, 0.9075148021937357, 0.3874205624614571, 0.05429633448473882], 85.89103055252225, 150.52822732611227, [0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19  …  0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9])

# Use Linear Regression to predict the optimal y_value

In [9]:
#Generating data
seed_number = 1
x_data,y_data,y_min_data,y_max_data,d_data = Data(n,m,alpha,beta,gamma,rho,sigma,form_data,seed_number)
com_data = hcat(x_data,y_data)
df = DataFrame(x_data = com_data[:,1],y_data = com_data[:,2])   # convert the data to DataFrame format
# Train test split
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.8)   # 80% of data points belong to the training set while 20% of them belong to the test set
#Train the model
lin_mol = @formula(y_data ~ x_data)
linearRegressor = lm(lin_mol, train)
# Prediction
ypredicted_test = predict(linearRegressor, test)
y_opti = maximum(ypredicted_test)   #The maximum value of linear regression

┌ Info: Precompiling Lathe [38d8eb38-e7b1-11e9-0012-376b6c802672]
└ @ Base loading.jl:1342
┌ Warning: Module DataFrames with build ID 286214177764100 is missing from the cache.
│ This may mean DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Info: Skipping precompilation since __precompile__(false). Importing Lathe [38d8eb38-e7b1-11e9-0012-376b6c802672].
└ @ Base loading.jl:1047


0.5879331053849443

In [11]:
#Using cross validation to predict
#Generating data
using MLDataUtils
seed_number = 1
x_data,y_data,y_min_data,y_max_data,d_data = Data(n,m,alpha,beta,gamma,rho,sigma,form_data,seed_number)
com_data = hcat(x_data,y_data)
df = DataFrame(x_data = com_data[:,1],y_data = com_data[:,2])   # convert the data to DataFrame format
# Train test split
fold = kfolds(df,5)   #Divide the data into 5 groups
y_opt = []
#Construct the linear regression
lin_mol = @formula(y_data ~ x_data)

for i = 1:5
    train = fold[i][1]
    test = fold[i][2]
    linearRegressor = lm(lin_mol, train)
    ypredicted_test = predict(linearRegressor, test)
    y_opti = maximum(ypredicted_test)
    y_opt = push!(y_opt,y_opti)
end

y_optivalue = mean(y_opt)
println("The average y optimal value is: ", y_optivalue)
    
#using Lathe.preprocess: TrainTestSplit
#train, test = TrainTestSplit(df,.8)   # 80% of data points belong to the training set while 20% of them belong to the test set
#Train the model
#lin_mol = @formula(y_data ~ x_data)
#linearRegressor = lm(lin_mol, train)
# Prediction
#ypredicted_test = predict(linearRegressor, test)
#y_opti = maximum(ypredicted_test)   #The maximum value of linear regression

The average y optimal value is: 0.5904872953536032


In [66]:
foldx = kfolds(df,5)
foldx[1]

(800×2 SubDataFrame
 Row │ x_data     y_data    
     │ Float64    Float64   
─────┼──────────────────────
   1 │ 0.477189   0.969435
   2 │ 0.637573   0.0505436
   3 │ 0.737179   0.11541
   4 │ 0.551901   0.032917
   5 │ 0.923498   0.194008
   6 │ 0.812854   0.153517
   7 │ 0.185426   0.464932
   8 │ 0.205388   0.525031
   9 │ 0.0881437  0.362347
  10 │ 0.674453   0.0978369
  11 │ 0.337942   0.740546
  ⋮  │     ⋮          ⋮
 791 │ 0.391561   0.813638
 792 │ 0.0864758  0.334206
 793 │ 0.24714    0.560925
 794 │ 0.886806   0.16632
 795 │ 0.291905   0.63365
 796 │ 0.519958   0.0614257
 797 │ 0.167934   0.489822
 798 │ 0.641344   0.0870546
 799 │ 0.592039   0.113178
 800 │ 0.730528   0.128169
            779 rows omitted, 200×2 SubDataFrame
 Row │ x_data      y_data    
     │ Float64     Float64   
─────┼───────────────────────
   1 │ 0.236033    0.596893
   2 │ 0.346517    0.705333
   3 │ 0.312707    0.676101
   4 │ 0.00790928  0.22625
   5 │ 0.488613    0.937746
   6 │ 0.210968    0.53

In [47]:
using MLDataUtils
folds = kfolds([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],5)
for i = 1:5
    println(folds[i])
end
println(folds[1][1])

([5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 2, 3, 4])
([1, 2, 3, 4, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [5, 6, 7, 8])
([1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 15, 16, 17, 18, 19, 20], [9, 10, 11, 12])
([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 18, 19, 20], [13, 14, 15, 16])
([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 20])
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


The optimal y_value obtained by linear regression is 0.5879331053849443.

In [30]:
a,b = coef(linearRegressor)
println("The coefficients of Linear Regression are: ",a ,b)

The coefficients of Linear Regression are: 0.5923808107514184-0.4395937695755845


# Use Quadratic Regression to predict the optimal y_value

In [31]:
# Obtain the square of the x_data
x_squ = x_data.^2   # Square every element in x_data
com_data = hcat(x_data,x_squ,y_data)
#com_data = com_data'   #The first column is x_data, the second column is y_data
df2 = DataFrame(x = com_data[:,1],x_squ = com_data[:,2],y_data = com_data[:,3])   # convert the data to DataFrame format
# Train test split
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df2,.8)   # 80% of data points belong to the training set while 20% of them belong to the test set
#Train the model
lin_mol = @formula(y_data ~ x+x_squ)
linearRegressor = lm(lin_mol, train)
# Prediction
ypredicted_test2 = predict(linearRegressor, test)
y_opti = maximum(ypredicted_test2)   #The maximum value of linear regression

0.5018067227988963

In [78]:
#Using cross validation to predict
#Generating data
using MLDataUtils
# Obtain the square of the x_data
x_squ = x_data.^2   # Square every element in x_data
com_data = hcat(x_data,x_squ,y_data)
df2 = DataFrame(x_data = com_data[:,1],x_squ = com_data[:,2],y_data = com_data[:,3])   # convert the data to DataFrame format
#println(df2)
# Train test split
fold = kfolds(df2,5)   #Divide the data into 5 groups
println(length(foldx))
println(length(foldy))
y_opt = []
#Construct the linear regression
lin_mol = @formula(y_data ~ x_data+x_squ)

for i = 1:5
    train = fold[i][1]
    test = fold[i][2]
    linearRegressor = lm(lin_mol, train)
    ypredicted_test = predict(linearRegressor, test)
    y_opti = maximum(ypredicted_test)
    y_opt = push!(y_opt,y_opti)
end

y_optivalue = mean(y_opt)
println("The average y optimal value is: ", y_optivalue)
    
#using Lathe.preprocess: TrainTestSplit
#train, test = TrainTestSplit(df,.8)   # 80% of data points belong to the training set while 20% of them belong to the test set
#Train the model
#lin_mol = @formula(y_data ~ x_data)
#linearRegressor = lm(lin_mol, train)
# Prediction
#ypredicted_test = predict(linearRegressor, test)
#y_opti = maximum(ypredicted_test)   #The maximum value of linear regression

5
5
The average y optimal value is: 0.5032377518600091


In [32]:
a,b,c = coef(linearRegressor)
println("The coefficients of Quadratic Regression are: ",a,b,c)

The coefficients of Quadratic Regression are: 0.41337493566375610.6149568858568986-1.069105332090855


The optimal y_value obtained by linear regression is 0.5156514336460036.

Since the y_value produced by EWO outperforms that of linear and quadratic models, we can conclude that EWO is significantly better than linear and quadratic ones.

In [11]:
#Initializing results table
Results = Array{Union{Missing, String, Int64, Float64}}(missing, 3*seed_numbers, 11+6)

seed_number = 1
#Running models for specified number of seeds to generate results
for seed_number = 1:seed_numbers
	println()
	println("seed_number: ", seed_number)

	#Initializing parameter settings in results table
	Results[3*(seed_number-1)+1,1:11] = [form_data,alpha,beta,gamma,rho,sigma,kappa,lambda,n,m,seed_number]
	Results[3*(seed_number-1)+2,1:11] = [form_data,alpha,beta,gamma,rho,sigma,kappa,lambda,n,m,seed_number]
	Results[3*(seed_number-1)+3,1:11] = [form_data,alpha,beta,gamma,rho,sigma,kappa,lambda,n,m,seed_number]

	#Generating data
	x_data,y_data,y_min_data,y_max_data,d_data = Data(n,m,alpha,beta,gamma,rho,sigma,form_data,seed_number)

	#Solving Estimate-While-Optimize model
	solution_EWO = EWO(x_data,y_data,d_data,kappa,lambda,big_M)

	#Saving solution and plotting parameters for Estimate-While-Optimize model
	a_EWO = solution_EWO[3]
	b_EWO = solution_EWO[4]
	z_EWO = solution_EWO[5]
	x_EWO = dot(d_data,z_EWO)
	y_EWO = Result(y_min_data,y_max_data,x_EWO,alpha,beta,gamma,rho,form_data)
	x_EWO_range = collect(x_EWO - kappa:(2*kappa/100):x_EWO + kappa)
	y_EWO_range = a_EWO*ones(length(x_EWO_range)) + b_EWO*x_EWO_range

	#Writing Estimate-While-Optimize results to results table
	Results[3*(seed_number-1)+1,12:17] = ["EWO",a_EWO,b_EWO,0,x_EWO[1],y_EWO[1]]

	#Printing Estimate-While-Optimize results
	println("EWO")
	println("a: ", a_EWO)
	println("b: ", b_EWO)
	println("x: ", x_EWO)
	println("y: ", y_EWO)

	#Solving Estimate-Then-Optimize-Linear model
	solution_ETOL = ETO(x_data,y_data,d_data,"linear")

	#Saving solution and plotting parameters for Estimate-Then-Optimize-Linear model
	a_ETOL = solution_ETOL[3]
	b_ETOL = solution_ETOL[4]
	c_ETOL = solution_ETOL[5]
	z_ETOL = solution_ETOL[6]
	x_ETOL = dot(d_data,z_ETOL)
	y_ETOL = Result(y_min_data,y_max_data,x_ETOL,alpha,beta,gamma,rho,form_data)
	x_ETOL_range = collect(0:1/100:1)
	y_ETOL_range = a_ETOL*ones(length(x_ETOL_range)) + b_ETOL*x_ETOL_range + c_ETOL*x_ETOL_range.^2

	#Writing Estimate-Then-Optimize-Linear results to results table
	Results[3*(seed_number-1)+2,12:17] = ["ETOL",a_ETOL,b_ETOL,c_ETOL,x_ETOL[1],y_ETOL[1]]

	#Printing Estimate-Then-Optimize-Linear results
	println("ETOL")
	println("a: ", a_ETOL)
	println("b: ", b_ETOL)
	println("c: ", c_ETOL)
	println("x: ", x_ETOL)
	println("y: ", y_ETOL)

	#Solving Estimate-Then-Optimize-Quadratic model
	solution_ETOQ = ETO(x_data,y_data,d_data,"quadratic")

	#Saving solution and plotting parameters for Estimate-Then-Optimize-Quadratic model
	a_ETOQ = solution_ETOQ[3]
	b_ETOQ = solution_ETOQ[4]
	c_ETOQ = solution_ETOQ[5]
	z_ETOQ = solution_ETOQ[6]
	x_ETOQ = dot(d_data,z_ETOQ)
	y_ETOQ = Result(y_min_data,y_max_data,x_ETOQ,alpha,beta,gamma,rho,form_data)
	x_ETOQ_range = collect(0:1/100:1)
	y_ETOQ_range = a_ETOQ*ones(length(x_ETOQ_range)) + b_ETOQ*x_ETOQ_range + c_ETOQ*x_ETOQ_range.^2

	#Writing Estimate-Then-Optimize-Quadratic results to results table
	Results[3*(seed_number-1)+3,12:17] = ["ETOQ",a_ETOQ,b_ETOQ,c_ETOQ,x_ETOQ[1],y_ETOQ[1]]

	#Printing Estimate-Then-Optimize-Quadratic results
	println("ETOQ")
	println("a: ", a_ETOQ)
	println("b: ", b_ETOQ)
	println("c: ", c_ETOQ)
	println("x: ", x_ETOQ)
	println("y: ", y_ETOQ)
end

#Writing results table to text file
writedlm("Results.txt", Results)
	
#Plotting data
gr()
#p = plot(x_data,y_data,seriestype=:scatter,title="Objective",xlabel="Decision",ylabel="Outcome",label="Data",legendfontsize=5,markersize=2,markercolor="grey")
p = plot!(x_EWO_range,y_EWO_range,label="EWO",linewidth=2,linecolor="red")
p = plot!(x_ETOL_range,y_ETOL_range,label="ETOL",linewidth=2,linecolor="blue")
p = plot!(x_ETOQ_range,y_ETOQ_range,label="ETOQ",linewidth=2,linecolor="green")
plot(p)


seed_number: 1
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.2371496928012201
b: 1.472283559654956
x: 0.43
y: 0.8706379893180601
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5924714138742307
b: -0.4565708145479483
c: 0
x: 0.1
y: 0.37007955550835164
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.4155542550964126
b: 0.6099740231183575
c: -1.0629647870791048
x: 0.29
y: 0.6582798658836383

seed_number: 2
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.21762895601327537
b: 1.5315618896585057
x: 0.45
y: 0.9067941946596493
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5973213057848756
b: -0.46628616777743526
c: 0
x: 0.1
y: 0.373031460821028
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.4186577867846067
b: 0.6146541444792838
c: -1.07794040941837
x: 0.29
y: 0.6627883734762796

seed_number: 3
Academic license 

seed_number: 18
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.21492389301962872
b: 1.5048928896514315
x: 0.45
y: 0.8906976969406624
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.58721119012154
b: -0.4644995881324681
c: 0
x: 0.1
y: 0.3629368169010162
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.40020234045350844
b: 0.668535631296823
c: -1.1366537899611924
x: 0.29
y: 0.6494355803511098

seed_number: 19
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.24392894918609842
b: 1.4417882392747303
x: 0.44
y: 0.8812283760132977
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5793918301650695
b: -0.471109166448303
c: 0
x: 0.1
y: 0.35844169386998864
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.40788562360160996
b: 0.574462864662235
c: -1.0520296639512277
x: 0.27
y: 0.6198350349416432

seed_number: 20
Academic licen

Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.25846951572658894
b: 1.4337864546076082
x: 0.37
y: 0.7895857998063344
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5866587541545705
b: -0.4558210900252632
c: 0
x: 0.1
y: 0.3775219048391824
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.42919940968403614
b: 0.48379797086016885
c: -0.9356796890839698
x: 0.26
y: 0.6217079166715688

seed_number: 36
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.18569360674267615
b: 1.6255873959631972
x: 0.39
y: 0.8175315344942506
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.6003873483055852
b: -0.4696267024477841
c: 0
x: 0.1
y: 0.37288890150117077
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.41299653666015734
b: 0.6694951241276589
c: -1.1373285783617308
x: 0.29
y: 0.6642064886345679

seed_number: 37
Academic license - for n

Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.16334610424650917
b: 1.6568456709666548
x: 0.39
y: 0.8095806519935811
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5943889560862695
b: -0.4761710848139299
c: 0
x: 0.1
y: 0.3677650332016595
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.41384025062581165
b: 0.6201356671956196
c: -1.1023376507234564
x: 0.28
y: 0.6419954172794039

seed_number: 53
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.2571117551120015
b: 1.4634091889553935
x: 0.42
y: 0.8711551706755238
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.6198835813323976
b: -0.5006355228939608
c: 0
x: 0.1
y: 0.3754472969480925
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.4283317817392821
b: 0.6351417118025902
c: -1.1397378069093906
x: 0.28
y: 0.6542829759197726

seed_number: 54
Academic license - for non-c

seed_number: 69
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.1919235024948691
b: 1.5694908419191218
x: 0.44
y: 0.884178429940014
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5962545055808179
b: -0.4854602684994252
c: 0
x: 0.1
y: 0.3620540505762874
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.4104039667356611
b: 0.6355083815891265
c: -1.1314724388487423
x: 0.28
y: 0.6384728396512015

seed_number: 70
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.22161626287548788
b: 1.5272383742599978
x: 0.45
y: 0.9099958866952177
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5985306052500825
b: -0.4699524075702367
c: 0
x: 0.1
y: 0.3724169638175045
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.4227667742478161
b: 0.5859029638215286
c: -1.0501966717912998
x: 0.28
y: 0.6488861241546141

seed_number: 71
Academic licen

seed_number: 86
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.1810644818343672
b: 1.609105171497272
x: 0.41
y: 0.8410492775765657
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5954059017462723
b: -0.4704136274229131
c: 0
x: 0.1
y: 0.36823566852918266
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.40737350786135273
b: 0.6755386902440692
c: -1.1528798719033717
x: 0.29
y: 0.6580246547195142

seed_number: 87
Academic license - for non-commercial use only - expires 2022-08-28
EWO
a: 0.21405088452976778
b: 1.5371554037642685
x: 0.42
y: 0.86101823105719
Academic license - for non-commercial use only - expires 2022-08-28
ETOL
a: 0.5974654590587006
b: -0.48932860646219106
c: 0
x: 0.1
y: 0.3722379188887473
Academic license - for non-commercial use only - expires 2022-08-28
ETOQ
a: 0.4386263761379937
b: 0.4602251780271559
c: -0.9583688499121724
x: 0.24
y: 0.586079305462441

seed_number: 88
Academic licen

LoadError: UndefVarError: x_data not defined

In [12]:
#Plotting data
gr()
#p = plot(x_data,y_data,seriestype=:scatter,title="Objective",xlabel="Decision",ylabel="Outcome",label="Data",legendfontsize=5,markersize=2,markercolor="grey")
p = plot!(x_EWO_range,y_EWO_range,label="EWO",linewidth=2,linecolor="red")
p = plot!(x_ETOL_range,y_ETOL_range,label="ETOL",linewidth=2,linecolor="blue")
p = plot!(x_ETOQ_range,y_ETOQ_range,label="ETOQ",linewidth=2,linecolor="green")
plot(p)

LoadError: UndefVarError: x_EWO_range not defined